In [1]:
# script.py
import os
import requests
import pandas as pd
import json
from time import sleep
import zipfile
from pymongo import MongoClient

# read in usda 

In [2]:
def get_client():
    client = MongoClient(host='usda_mongodb',
                         port=27017, 
                         username='root', 
                         password='rootpassword',
                        authSource="admin")
    return client

In [3]:
#usda_json_files = {'FoundationFoods': './data/usda/unzipped/FoodData_Central_foundation_food_json_2022-10-28.json',
#                  'BrandedFoods': './data/usda/unzipped/FoodData_Central_branded_food_json_2022-10-28.json',
#                  'SRLegacyFoods': './data/usda/unzipped/FoodData_Central_sr_legacy_food_json_2021-10-28.json',
#                  'SurveyFoods': './data/usda/unzipped/FoodData_Central_survey_food_json_2022-10-28.json'}

usda_json_files = {'FoundationFoods': './data/usda/FoodData_Central_foundation_food_json_2022-10-28.json',
                    'BrandedFoods': './data/usda/FoodData_Central_branded_food_json_2022-10-28.json'}
'''
# drop collection
client = get_client()
with client:
    db = client.usda_foods
    for f in ['BrandedFoods']:
        collection = db[f]
        print(collection.count_documents({}))
        collection.drop()
       
# create collection
client = get_client()
with client:
    db = client.usda_foods
    for k, v in usda_json_files.items():
        with open(v) as json_file:
            data = json.load(json_file)
            db[k].insert_many(data[k])
    print(db.list_collection_names())
    
client = get_client()
with client:
    db = client.usda_foods
    for f in ['BrandedFoods']:
        collection = db[f]
        print(collection.count_documents({}))
''' 

"\n# drop collection\nclient = get_client()\nwith client:\n    db = client.usda_foods\n    for f in ['BrandedFoods']:\n        collection = db[f]\n        print(collection.count_documents({}))\n        collection.drop()\n       \n# create collection\nclient = get_client()\nwith client:\n    db = client.usda_foods\n    for k, v in usda_json_files.items():\n        with open(v) as json_file:\n            data = json.load(json_file)\n            db[k].insert_many(data[k])\n    print(db.list_collection_names())\n    \nclient = get_client()\nwith client:\n    db = client.usda_foods\n    for f in ['BrandedFoods']:\n        collection = db[f]\n        print(collection.count_documents({}))\n"

In [4]:
# list databases
client = get_client()
for db in client.list_databases():
    print(db)

{'name': 'admin', 'sizeOnDisk': 102400, 'empty': False}
{'name': 'config', 'sizeOnDisk': 73728, 'empty': False}
{'name': 'local', 'sizeOnDisk': 77824, 'empty': False}
{'name': 'mydatabase', 'sizeOnDisk': 40960, 'empty': False}
{'name': 'usda_foods', 'sizeOnDisk': 516575232, 'empty': False}


In [5]:
# create collection
client = get_client()
with client:
    db = client.usda_foods
    #db.foundation_foods.insert_many(data['FoundationFoods'])
    #db.foundation_foods.insert_many(data['BrandedFoods'])
    #db.foundation_foods.insert_many(data['SRLegacyFoods'])
    #db.foundation_foods.insert_many(data['SurveyFoods'])
    print(db.list_collection_names())

['BrandedFoods', 'FoundationFoods']


In [6]:
# list collections
client = get_client()
with client:
    db = client.usda_foods
    for f in ['BrandedFoods', 'SRLegacyFoods', 'SurveyFoods', 'FoundationFoods']:
        print(f)
        collection = db[f]
        #collection.drop()
        # count documents in collection
        print(collection.count_documents({}))
        limit_rows = 0
        for studentinfo in collection.find():
            print(studentinfo['fdcId'])
            print(studentinfo.keys())
            break
            limit_rows += 1
            if limit_rows == 2:
                break

BrandedFoods
393327
1106281
dict_keys(['_id', 'foodClass', 'description', 'foodNutrients', 'foodAttributes', 'modifiedDate', 'availableDate', 'marketCountry', 'brandOwner', 'gtinUpc', 'dataSource', 'ingredients', 'servingSize', 'servingSizeUnit', 'labelNutrients', 'tradeChannels', 'microbes', 'brandedFoodCategory', 'publicationDate', 'foodUpdateLog', 'fdcId', 'dataType'])
SRLegacyFoods
0
SurveyFoods
0
FoundationFoods
210
321358
dict_keys(['_id', 'foodClass', 'description', 'foodNutrients', 'foodAttributes', 'nutrientConversionFactors', 'isHistoricalReference', 'ndbNumber', 'fdcId', 'dataType', 'foodCategory', 'inputFoods', 'foodPortions', 'publicationDate'])


In [7]:
# query doc in collection
client = get_client()
with client:
    db = client.usda_foods
    collection = db['FoundationFoods']
    #print(collection.find_one())
    #print(collection.find_one({}, {"description": "Hummus, commercial"}))
    print(collection.find({"fdcId": {"$eq": 321358}}))
    myquery = { "fdcId": 321358}
    mydoc = collection.find(myquery)

    for x in mydoc:
      print(x.keys())

dict_keys(['_id', 'foodClass', 'description', 'foodNutrients', 'foodAttributes', 'nutrientConversionFactors', 'isHistoricalReference', 'ndbNumber', 'fdcId', 'dataType', 'foodCategory', 'inputFoods', 'foodPortions', 'publicationDate'])


In [8]:
# bring in nutrients
nutrient_list_df = pd.read_csv('./data/nutrient_list.csv')
print(nutrient_list_df.shape)
nutrient_list_df = nutrient_list_df.fillna(0)
nutrient_list_df['nutrientId'] = pd.to_numeric(nutrient_list_df['nutrientId'], errors='coerce')
nutrient_list_df['nutrientId'] = nutrient_list_df['nutrientId'].astype('int')
nutrient_list_df = nutrient_list_df[nutrient_list_df['nutrientId'].astype('str').str.len() > 1]
nutrient_list_df['nutrientId'] = nutrient_list_df['nutrientId'].astype('str')
print(nutrient_list_df.shape)
nutrient_list_df.head(1)

(191, 2)
(190, 2)


,nutrientId,nutrientName
0,328,Vitamin D (D2 + D3)


In [9]:
nutrients_of_interest = {}
for key, row in nutrient_list_df.iterrows():
    nutrients_of_interest[row['nutrientId']] = row['nutrientName']

In [10]:
def get_legacy_food_info(food_id_json):
    #print(food_id_json.keys())
    #print(food_id_json)
    general_cols_interest = ['fdcId', 'foodClass', 'description', 'brandOwner',
                             'brandedFoodCategory', 'foodCategory', 'servingSize','servingSizeUnit', 'dataType', 'ingredients']
    general_cols_dict = {}
    for g in general_cols_interest:
        general_cols_dict[g] = food_id_json.get(g, None)
    '''
    labelNutrients = food_id_json.get('labelNutrients', None)
    for k,v in labelNutrients.items():
        general_cols_dict[k] = v.get('value', None)
    '''
    food_nutrients_list = food_id_json.get('foodNutrients', None)
    for nutrient in food_nutrients_list:
        for k,v in nutrients_of_interest.items():
            for nutrient in food_nutrients_list:
                nutrient_number = nutrient['nutrient'].get('number', None)
                if nutrient_number  == k:
                    food_id_label = 'nutrient_id_'+ k
                    general_cols_dict[k] = nutrient['amount']
    df = pd.DataFrame(general_cols_dict, index=[0])
    
    return df

In [11]:
'''
nutrients_of_interest = {'203': 'Protein', '204': 'Total lipid (fat)', '205': 'Carbohydrate, by difference',
                         '208':'Energy', '324':'Vitamin D (D2 + D3), International Units', '291': 'Fiber, total dietary',
                        '301': 'Calcium, Ca'}
'''
#dict_keys(['_id', 'foodClass', 'description', 'foodNutrients', 'foodAttributes', 'nutrientConversionFactors',
#'isHistoricalReference', 'ndbNumber', 'fdcId', 'dataType', 'foodCategory', 'inputFoods', 'foodPortions', 'publicationDate'])

# dict_keys(['_id', 'foodClass', 'description', 'foodNutrients', 'foodAttributes', 'modifiedDate', 'availableDate', 'marketCountry', 'brandOwner', 'gtinUpc', 'dataSource', 'ingredients', 
# 'servingSize','servingSizeUnit', 'labelNutrients', 'tradeChannels', 'microbes', 'brandedFoodCategory', 'publicationDate', 'foodUpdateLog', 'fdcId', 'dataType'])


labelNutrientsRequired = {'labelNutrients': ['fat', 'saturatedFat', 'transFat', 'cholesterol', 'sodium', 'carbohydrates', 'fiber', 'sugars', 'protein', 'calcium', 'iron', 'calories']}

def get_specific_food_info(food_id_json):
    #print(food_id_json.keys())
    #print(food_id_json)
    general_cols_interest = ['fdcId', 'foodClass', 'description', 'brandOwner',
                             'brandedFoodCategory',  'servingSize','servingSizeUnit', 'dataType', 'ingredients']
    general_cols_dict = {}
    for g in general_cols_interest:
        general_cols_dict[g] = food_id_json.get(g, None)
    try:
        general_cols_dict['foodCategory'] = food_id_json.get('foodCategory', None)['description']
    except:
        general_cols_dict['foodCategory'] = food_id_json.get('foodCategory', None)
    
    labelNutrients = food_id_json.get('labelNutrients', None)
    try:
        for k,v in labelNutrients.items():
            general_cols_dict[k] = v.get('value', None)
    except:
        pass
    
    for index, row in nutrient_list_df.iterrows():
        nutrient_cols_interest = ['nutrientId', 'nutrientName', 'nutrientNumber', 'unitName', 'derivationCode', 'derivationDescription',
                                 'derivationId', 'value', 'foodNutrientSourceId', 'foodNutrientSourceCode', 'FoodNutrientSourceDescription',
                                 'rank']
        
        foodNutrients = food_id_json.get('foodNutrients', None)
        for n in foodNutrients:
            if str(n['nutrient']['number']) == str(row['nutrientId']):
                general_cols_dict[row['nutrientId']] = n.get('amount', None)

    df = pd.DataFrame(general_cols_dict, index=[0])
    # add in blank nutrients
    for n in nutrient_list_df['nutrientId']:
        if n not in df.columns:
            df[n] = None
            
    # add in blank label nutrients
    for l in labelNutrientsRequired['labelNutrients']:
        if l not in df.columns:
            df[l] = None
    return df

In [ ]:
#fdcid_list_df = pd.read_csv('./data/fdcid_list.csv')
all_food_df = pd.DataFrame()
#num_of_ids = len(fdcid_list_df['fdc_id'])

client = get_client()
with client:
    db = client.usda_foods
    #for f in ['BrandedFoods', 'SRLegacyFoods', 'SurveyFoods', 'FoundationFoods']:
    #for f in ['FoundationFoods']:
    for f in ['BrandedFoods', 'FoundationFoods']:
        collection = db[f]
        limit_rows = 0
        count_api_calls = 0 
        num_of_ids = collection.count_documents({})
        for food_info in collection.find(no_cursor_timeout=True, batch_size=1):
            if food_info is not None:
                #print(food_info['fdcId'])
                limit_rows += 1
                df = get_specific_food_info(food_info)
                #all_food_df = all_food_df.append(df)
                all_food_df = pd.concat((all_food_df, df), axis=0)
                count_api_calls = count_api_calls + 1 
                if count_api_calls % 1000 == 0:
                    print(count_api_calls, 'rows of', num_of_ids)
            #if count_api_calls == 1:
            #    break
print(all_food_df.shape)
all_food_df.to_csv('./data/all_foods.csv', index=False)
all_food_df.head()

/tmp/ipykernel_52/3782478824.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[n] = None
/tmp/ipykernel_52/3782478824.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[l] = None


1000 rows of 393327
2000 rows of 393327
3000 rows of 393327
4000 rows of 393327
5000 rows of 393327
6000 rows of 393327
7000 rows of 393327
8000 rows of 393327
9000 rows of 393327
10000 rows of 393327
11000 rows of 393327
12000 rows of 393327
13000 rows of 393327
14000 rows of 393327
15000 rows of 393327
16000 rows of 393327
17000 rows of 393327
18000 rows of 393327
19000 rows of 393327
20000 rows of 393327
21000 rows of 393327
22000 rows of 393327
23000 rows of 393327
24000 rows of 393327
25000 rows of 393327
26000 rows of 393327
27000 rows of 393327
28000 rows of 393327
29000 rows of 393327
30000 rows of 393327
31000 rows of 393327
32000 rows of 393327
33000 rows of 393327
34000 rows of 393327
35000 rows of 393327
36000 rows of 393327
37000 rows of 393327
38000 rows of 393327
39000 rows of 393327
40000 rows of 393327
41000 rows of 393327
42000 rows of 393327
43000 rows of 393327
44000 rows of 393327
45000 rows of 393327
46000 rows of 393327
47000 rows of 393327
48000 rows of 393327
4

In [128]:
all_food_df['291']

0    7.1
Name: 291, dtype: float64

In [4]:
api_key = os.environ.get('USDA_API_KEY')
api_key

'BsgugxamECM2YaBylT5QNDOaNGAP8XHS93H8dbFt'

## Read in all foods from CSV

In [6]:
'''
branded_food_df = pd.read_csv('./data/food_data_central/branded_food.csv')
branded_food_df['dataType'] = 'Branded'
print(branded_food_df.shape)

foundation_food_df = pd.read_csv('./data/food_data_central/foundation_food.csv')
foundation_food_df['dataType'] = "Foundation"
print(foundation_food_df.shape)

sr_legacy_food_df = pd.read_csv('./data/food_data_central/sr_legacy_food.csv')
sr_legacy_food_df['dataType'] = "SR Legacy"
print(sr_legacy_food_df.shape)

frames = [branded_food_df[['fdc_id', 'dataType']], foundation_food_df[['fdc_id', 'dataType']], sr_legacy_food_df[['fdc_id', 'dataType']]]
fdcid_list_df = pd.concat(frames)
fdcid_list_df.to_csv('./data/fdcid_list.csv', index=False)
print(fdcid_list_df.shape)
fdcid_list_df.head(1)
'''

'\nbranded_food_df = pd.read_csv(\'./data/food_data_central/branded_food.csv\')\nbranded_food_df[\'dataType\'] = \'Branded\'\nprint(branded_food_df.shape)\n\nfoundation_food_df = pd.read_csv(\'./data/food_data_central/foundation_food.csv\')\nfoundation_food_df[\'dataType\'] = "Foundation"\nprint(foundation_food_df.shape)\n\nsr_legacy_food_df = pd.read_csv(\'./data/food_data_central/sr_legacy_food.csv\')\nsr_legacy_food_df[\'dataType\'] = "SR Legacy"\nprint(sr_legacy_food_df.shape)\n\nframes = [branded_food_df[[\'fdc_id\', \'dataType\']], foundation_food_df[[\'fdc_id\', \'dataType\']], sr_legacy_food_df[[\'fdc_id\', \'dataType\']]]\nfdcid_list_df = pd.concat(frames)\nfdcid_list_df.to_csv(\'./data/fdcid_list.csv\', index=False)\nprint(fdcid_list_df.shape)\nfdcid_list_df.head(1)\n'

In [8]:
#fdcid_list_df = pd.read_csv('./data/fdcid_list.csv')
#fdcid_list_df.shape

## Get nutrition facts from API Food List

In [7]:
def get_multiple_foods(food_id_list, api_key):
    data = {'api_key': api_key}
    #headers={'x-api-key':api_key, "Content-Type":"application/json"}
    #payload = {'query': 'beef', 'dataType': 'Foundataion', 'pageSize': 1}
    #https://api.nal.usda.gov/fdc/v1/foods?fdcIds=%22534358%22&fdcIds=%22373052%22&format=full'
    #url = f'https://api.nal.usda.gov/fdc/v1/foods?fdcIds=%22534358%22&fdcIds=%22373052%22&format=full?api_key={api_key}'
    #https://api.nal.usda.gov/fdc/v1/foods?fdcIds=%22534358%22&format=full
    #food_id = str(167512)
    #food_id = str('167512,167513')
    food_id = ''
    for f in food_id_list:
        food_id = food_id  + str(f) + ','
    food_id = food_id.strip(',')
    print(food_id)
    #url = f'https://api.nal.usda.gov/fdc/v1/foods?fdcIds=%22534358%22&format=full'
    #url = 'https://api.nal.usda.gov/fdc/v1/foods?fdcIds=%22534358%22&format=full'
    url = f'https://api.nal.usda.gov/fdc/v1/foods?api_key=BsgugxamECM2YaBylT5QNDOaNGAP8XHS93H8dbFt&fdcIds={food_id}'
    # ?format=json
    #url = f'https://api.nal.usda.gov/fdc/v1/foods/search?api_key={apiKey}&query={foodName}'
    print(url)
    r = requests.get(url)
    if r.status_code == 200:
        return r.json()
    else:
        print(r.status_code)
        return None

In [8]:
#f_lst = [167512, 167513, 1105904, 1900168]
f_lst = [167512, 167513, 1105904, 1900168]
f_lst = fdcid_list_df['fdc_id'].to_list()[0:2] # 1105904
food_list_response = get_multiple_foods(f_lst, api_key)
for f in food_list_response:
    print(f['fdcId'])
#food_list_response

1105904,1105905
https://api.nal.usda.gov/fdc/v1/foods?api_key=BsgugxamECM2YaBylT5QNDOaNGAP8XHS93H8dbFt&fdcIds=1105904,1105905
1105904
1105905


In [9]:
def get_food(food_id, api_key):
    data = {"query" : food_id}
    url = f'https://api.nal.usda.gov/fdc/v1/food/{food_id}?api_key={api_key}'
    r = requests.get(url)
    if r.status_code == 200:
        return r.json()
    else:
        return None

In [10]:
food_id_json = get_food(1105904, api_key)
#food_id_json

In [64]:
nutrient_list_df = pd.read_csv('./data/nutrient_list.csv')
print(nutrient_list_df.shape)
nutrient_list_df = nutrient_list_df.fillna(0)
nutrient_list_df['nutrientId'] = pd.to_numeric(nutrient_list_df['nutrientId'], errors='coerce')
nutrient_list_df['nutrientId'] = nutrient_list_df['nutrientId'].astype('int')
nutrient_list_df = nutrient_list_df[nutrient_list_df['nutrientId'].astype('str').str.len() > 1]
nutrient_list_df['nutrientId'] = nutrient_list_df['nutrientId'].astype('str')
print(nutrient_list_df.shape)
nutrient_list_df.head(1)

(191, 2)
(190, 2)


,nutrientId,nutrientName
0,328,Vitamin D (D2 + D3)


In [65]:
nutrients_of_interest = {}
for key, row in nutrient_list_df.iterrows():
    nutrients_of_interest[row['nutrientId']] = row['nutrientName']

In [13]:
'''
nutrients_of_interest = {'203': 'Protein', '204': 'Total lipid (fat)', '205': 'Carbohydrate, by difference',
                         '208':'Energy', '324':'Vitamin D (D2 + D3), International Units', '291': 'Fiber, total dietary',
                        '301': 'Calcium, Ca'}
'''
def get_food_info(food_id_json):
    general_cols_interest = ['fdcId', 'description', 'dataType', 'foodClass','brandOwner', 'brandName',
                             'brandedFoodCategory','servingSize','servingSizeUnit']
    general_cols_dict = {}
    for g in general_cols_interest:
        general_cols_dict[g] = food_id_json.get(g, None)
        
    food_category = food_id_json.get('foodCategory', None)
    for f in food_category:
        if f == 'description':
            cat_desc = food_category.get(f, None)
            general_cols_dict['food_category_description'] = cat_desc
    food_nutrients_list = food_id_json.get('foodNutrients', None)
    for nutrient in food_nutrients_list:
        for k,v in nutrients_of_interest.items():
            for nutrient in food_nutrients_list:
                nutrient_number = nutrient['nutrient'].get('number', None)
                if nutrient_number  == k:
                    food_id_label = 'nutrient_id_'+ k
                    general_cols_dict[k] = nutrient['amount']

    # add in blank nutrients
    for k,v in nutrients_of_interest.items():
        if k not in general_cols_dict.keys():
            general_cols_dict[k] = None
    df = pd.DataFrame(general_cols_dict, index=[0])
    return df

In [32]:
#fdcid_list_df = pd.read_csv('./data/fdcid_list.csv')
all_food_df = pd.DataFrame()
count_api_calls = 0 
num_of_ids = len(fdcid_list_df['fdc_id'])
#for f in fdcid_list_df['fdc_id']:
for f in food_list_response:
    #print(f)
    #break
    #food_id_json = get_food(f, api_key)
    food_id_json = f
    if food_id_json is not None:
        df = get_food_info(food_id_json)
        all_food_df = all_food_df.append(df)
        count_api_calls = count_api_calls + 1 
        if count_api_calls % 100 == 0:
            print(count_api_calls, 'rows of', num_of_ids)
            sleep(1)  # Time in seconds

print(all_food_df.shape)
all_food_df.to_csv('./data/all_foods.csv', index=False)
all_food_df.head()

TypeError: 'NoneType' object is not iterable

## Contact API with Page Numbers

In [20]:
dataType = 'Branded'
pageNumber = '200'
pageSize = '50'
url = f'https://api.nal.usda.gov/fdc/v1/foods/list?api_key={api_key}&dataType={dataType}&pageNumber={pageNumber}&pageSize={pageSize}'
r = requests.get(url)
print(r.status_code, len(r.json()), r)

dataType = 'Branded'
pageNumber = '201'
pageSize = '50'
url = f'https://api.nal.usda.gov/fdc/v1/foods/list?api_key={api_key}&dataType={dataType}&pageNumber={pageNumber}&pageSize={pageSize}'
r = requests.get(url)
print(r.status_code, len(r.json()), r)

200 50 <Response [200]>
500 5 <Response [500]>


In [21]:
def get_all_food_fdcId(api_key, dataType, pageSize, pageNumber):
    fdcid_list = []
    data = {'dataType': [dataType],'pageSize':pageSize, 'pageNumber': pageNumber}
    #data = {'dataType': dataType, 'pageSize':pageSize, 'pageNumber': pageNumber}
    #data = {"query" : foodName, 'dataType': ,
    print(data)
    url = f'https://api.nal.usda.gov/fdc/v1/foods/list?api_key={api_key}&dataType={dataType}&pageNumber={pageNumber}'
    # curl -X 'GET' \'https://api.nal.usda.gov/fdc/v1/foods/list?dataType=Branded&pageSize=25&pageNumber=2' \
      #-H 'accept: application/json'
    r = requests.get(url)
    #print(data, r.status_code, len(r.json()))
    try:
        assert r.status_code == 200, 'invalid'
        r_json = r.json()
        for f in r_json:
            fdcid_list.append(f.get('fdcId', None))
        return fdcid_list
    except AssertionError as msg:
        return msg

#food_list = get_all_food_fdcId(api_key, 'Foundation', 200, 100)

final_list = []                
dataTypes = ['Foundation', "SR Legacy", "Branded"]
for d in dataTypes:
    #food_list = get_all_food_fdcId(api_key, d, 200, 51)
    for i in range(1,10000):   
        food_list = get_all_food_fdcId(api_key, d, 200, i)
        if type(food_list) is list:
            final_list = final_list + food_list
        else:
            print('no more food list', d, i)
            break

print(len(final_list))

{'dataType': ['Foundation'], 'pageSize': 200, 'pageNumber': 1}
{'dataType': ['Foundation'], 'pageSize': 200, 'pageNumber': 2}
{'dataType': ['Foundation'], 'pageSize': 200, 'pageNumber': 3}
{'dataType': ['Foundation'], 'pageSize': 200, 'pageNumber': 4}
{'dataType': ['Foundation'], 'pageSize': 200, 'pageNumber': 5}
{'dataType': ['Foundation'], 'pageSize': 200, 'pageNumber': 6}
{'dataType': ['Foundation'], 'pageSize': 200, 'pageNumber': 7}
{'dataType': ['Foundation'], 'pageSize': 200, 'pageNumber': 8}
{'dataType': ['Foundation'], 'pageSize': 200, 'pageNumber': 9}
{'dataType': ['Foundation'], 'pageSize': 200, 'pageNumber': 10}
{'dataType': ['Foundation'], 'pageSize': 200, 'pageNumber': 11}
{'dataType': ['Foundation'], 'pageSize': 200, 'pageNumber': 12}
{'dataType': ['Foundation'], 'pageSize': 200, 'pageNumber': 13}
{'dataType': ['Foundation'], 'pageSize': 200, 'pageNumber': 14}
{'dataType': ['Foundation'], 'pageSize': 200, 'pageNumber': 15}
{'dataType': ['Foundation'], 'pageSize': 200, 'pa

In [22]:
len(final_list)

fdcid_list_df = pd.DataFrame(final_list, columns=['fdcId']).sort_values(['fdcId'])
fdcid_list_df.to_csv('./helpers/fdcId_list.csv', index=False)
print(fdcid_list_df.shape)
print('was 18k lne')
fdcid_list_df.head(5)

(18003, 1)
was 18k lne


,fdcId
5725,167512
5729,167513
4447,167514
4108,167515
7831,167516


In [20]:
# get all list
def get_all_food_fdcId(api_key):
    fdcid_list = []
    #data = {'dataType': [dataType],'pageSize':pageSize, 'pageNumber': pageNumber}
    #data = {'dataType': dataType, 'pageSize':pageSize, 'pageNumber': pageNumber}
    #data = {"query" : foodName, 'dataType': ,
    # https://api.nal.usda.gov/fdc/v1/foods/list?dataType=Foundation,SR%20Legacy&pageSize=200&pageNumber=1&sortBy=lowercaseDescription.keyword&sortOrder=asc
    url = f'https://api.nal.usda.gov/fdc/v1/foods/list?api_key={api_key}'
    # curl -X 'GET' \'https://api.nal.usda.gov/fdc/v1/foods/list?dataType=Branded&pageSize=25&pageNumber=2' \
      #-H 'accept: application/json'
    r = requests.get(url)
    #print(data, r.status_code, len(r.json()))
    try:
        assert r.status_code == 200, 'invalid'
        r_json = r.json()
        with open('./data/all_food_query.json', 'w') as f:
            json.dump(r_json, f)
        #for f in r_json:
        #    fdcid_list.append(f.get('fdcId', None))
        #return fdcid_list
    except AssertionError as msg:
        return msg
get_all_food_fdcId(api_key)

## search for food

In [4]:
url = 'https://api.nal.usda.gov/fdc/v1/foods/search'
headers={'x-api-key':api_key, "Content-Type":"application/json"}
payload = {'query': 'beef', 'dataType': 'Foundataion', 'pageSize': 1}

resp = requests.post(url, headers= headers, json=payload)
print(resp.status_code) 
print(len(resp.json()))
print(type(resp.json()))
print(resp.json().keys())
#resp.json()['foods']
#print(resp.json()['totalHits'])
resp.text

400
5
<class 'dict'>
dict_keys(['timestamp', 'status', 'error', 'message', 'path'])


'{"timestamp":"2022-12-06T22:18:28.854+0000","status":400,"error":"Bad Request","message":"Invalid request parameter or body","path":"/portal-data/api/v1/foods/search"}'

In [171]:
def parse_food_list(food_list_json, nutrient_list_df):
    #print(food_list_json)
    #food_list = ans['foods']
    print('number of food in list', len(food_list))
    food_name_id_dict = {}
    food_nutrient_dict = {}
    nutrientName_list = []
    df = pd.DataFrame()
    for f in food_list_json:
        #print(f['fdcId'])
        cols_interest = ['fdcId', 'description', 'dataType', 'brandOwner', 'brandName', 'foodCategory', 'servingSizeUnit', 'servingSize']
        for c in cols_interest:
            food_name_id_dict[c] = f.get(c, None)
        
        #print(food_name_id_dict, '\n')
        nutrients_of_interest = {'203': 'Protein', '204': 'Total lipid (fat)', '205': 'Carbohydrate, by difference', '208':'Energy'}
        for k,v in nutrients_of_interest.items():
            food_nutrients_list = f.get('foodNutrients', None)
            for nutrient in food_nutrients_list:
                if nutrient.get('number', None) == k:
                    food_id_label = 'nutrient_id_'+ k
                    food_name_id_dict[food_id_label] = nutrient['amount']
                


            '''          
            for index, row in nutrient_list_df.iterrows():
                nutrientId = row['nutrientId']
                nutrientName = row['nutrientName']
                if nutrient['number'] in list(nutrient_list_df['nutrientId']):
                    food_name_id_dict[nutrientId] = nutrient.get(nutrient['number'], None)'''
                
        df = df.append(pd.DataFrame(food_name_id_dict, index=[0]))
            
    return df

In [172]:
def get_food_list(api_key, dataType, pageSize, pageNumber):
    data = {'dataType': [dataType],'pageSize':pageSize, 'pageNumber': pageNumber}
    #data = {"query" : foodName, 'dataType': ['Foundation', "SR Legacy", "Branded"],
    url = f'https://api.nal.usda.gov/fdc/v1/foods/list?api_key={api_key}'
    r = requests.post(url, json=data)
    print('status code', r.status_code)  # 200
    return r.json()

#len(get_food_list(api_key, 'Foundation'))
nutrient_list_df = pd.read_csv('./helpers/nutrient_list.csv').head(191)
nutrient_list_df = nutrient_list_df.fillna(0)
nutrient_list_df['nutrientId'] = nutrient_list_df['nutrientId'].astype('int').astype('str')
print(nutrient_list_df.shape)
food_list = get_food_list(api_key, 'Branded', 2, 1)
print(len(food_list))
df = parse_food_list(food_list, nutrient_list_df)
df

(191, 2)
status code 200
2
number of food in list 2


,fdcId,description,dataType,brandOwner,brandName,foodCategory,servingSizeUnit,servingSize,nutrient_id_203,nutrient_id_204,nutrient_id_205,nutrient_id_208
0,1900168,"ALMOND MILK, ORIGINAL",Branded,H E Butt Grocery Company,None,None,None,None,0.42,1.04,3.33,25.0
0,1944909,"ARTISAN SMOOTH & CREAMY GELATO, TOASTED COCONUT",Branded,Lunds Inc,None,None,None,None,4.72,10.40,25.50,198.0


In [ ]:
single_food_json = get_food(321358, api_key)
print(single_food_json)
with open('./data/single_food.json', 'w') as f:
    json.dump(single_food_json, f)

In [59]:
def call_API_2_foundation(foodName, apiKey):
    data = {"query" : foodName, 'dataType': ['Foundation']}
    #data = {"query" : foodName, 'dataType': ['Foundation', "SR Legacy", "Branded"],
    url = f'https://api.nal.usda.gov/fdc/v1/foods/search?api_key={apiKey}'
    r = requests.post(url, json=data)
    print(r.status_code)  # 200
    return r.json()

ans = call_API_2_foundation("beef", "BsgugxamECM2YaBylT5QNDOaNGAP8XHS93H8dbFt")
print(ans.keys())

print(len(ans))
food_list = ans['foods']
food_name_id_dict = {}
food_nutrient_dict = {}
nutrientName_list = []
df = pd.DataFrame()
for f in food_list:
    print(f['fdcId'])
    cols_interest = ['fdcId', 'description', 'dataType', 'brandOwner', 'brandName', 'foodCategory', 'servingSizeUnit', 'servingSize']
    for c in cols_interest:
        food_name_id_dict[c] = f.get(c, None)
    df = df.append(pd.DataFrame(food_name_id_dict, index=[0]))
    #print(food_name_id_dict, '\n')
    food_nutrients_list = f.get('foodNutrients', None)
    #print('number of nutrient for', f['fdcId'], len(food_nutrients_list))
    for nutrient in food_nutrients_list:
        nutrient_cols_interest = ['nutrientId', 'nutrientName', 'nutrientNumber', 'unitName', 'derivationCode', 'derivationDescription',
                                 'derivationId', 'value', 'foodNutrientSourceId', 'foodNutrientSourceCode', 'FoodNutrientSourceDescription',
                                 'rank']
        if nutrient.get('nutrientName') == 'Carbohydrate, by difference':
            for n in nutrient_cols_interest:
                food_nutrient_dict['fdcId'] = f.get('fdcId', None)
                food_nutrient_dict[n] = nutrient.get(n, None)
                food_name_id_dict[n] = nutrient.get(n, None)
                
                nutrientName_list.append(nutrient.get('nutrientName'))
    #break
pd.DataFrame(food_name_id_dict, index=[0])
df

200
dict_keys(['totalHits', 'currentPage', 'totalPages', 'pageList', 'foodSearchCriteria', 'foods', 'aggregations'])
7
323121
746779
746758
746760
746761
746762
746759
746763


,fdcId,description,dataType,brandOwner,brandName,foodCategory,servingSizeUnit,servingSize,nutrientId,nutrientName,nutrientNumber,unitName,derivationCode,derivationDescription,derivationId,value,foodNutrientSourceId,foodNutrientSourceCode,FoodNutrientSourceDescription,rank
0,323121,"Frankfurter, beef, unheated",Foundation,None,None,Sausages and Luncheon Meats,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,746779,"Sausage, breakfast sausage, beef, pre-cooked, ...",Foundation,None,None,Sausages and Luncheon Meats,None,None,1005.0,"Carbohydrate, by difference",205,G,NC,Calculated,49.0,2.89,2.0,4,None,1110.0
0,746758,"Beef, loin, tenderloin roast, separable lean o...",Foundation,None,None,Beef Products,None,None,1005.0,"Carbohydrate, by difference",205,G,NC,Calculated,49.0,3.37,2.0,4,None,1110.0
0,746760,"Beef, round, eye of round roast, boneless, sep...",Foundation,None,None,Beef Products,None,None,1005.0,"Carbohydrate, by difference",205,G,AS,Summed,4.0,0.00,1.0,1,None,1110.0
0,746761,"Beef, round, top round roast, boneless, separa...",Foundation,None,None,Beef Products,None,None,1005.0,"Carbohydrate, by difference",205,G,AS,Summed,4.0,0.00,1.0,1,None,1110.0
0,746762,"Beef, short loin, porterhouse steak, separable...",Foundation,None,None,Beef Products,None,None,1005.0,"Carbohydrate, by difference",205,G,AS,Summed,4.0,0.00,1.0,1,None,1110.0
0,746759,"Beef, loin, top loin steak, boneless, lip-on, ...",Foundation,None,None,Beef Products,None,None,1005.0,"Carbohydrate, by difference",205,G,AS,Summed,4.0,0.00,1.0,1,None,1110.0
0,746763,"Beef, short loin, t-bone steak, bone-in, separ...",Foundation,None,None,Beef Products,None,None,1005.0,"Carbohydrate, by difference",205,G,AS,Summed,4.0,0.00,1.0,1,None,1110.0


In [113]:
def call_API_2(foodName, apiKey):
    #data = {"query" : foodName, 'dataType': ['Foundation'],
    data = {"query" : foodName, 'dataType': ['Foundation', "SR Legacy", "Branded"],
                                            'pageNumber': 2}
    url = f'https://api.nal.usda.gov/fdc/v1/foods/search?api_key={apiKey}'
    r = requests.post(url, json=data)
    print(r.status_code)  # 200
    return r.json()

ans = call_API_2("milk", "BsgugxamECM2YaBylT5QNDOaNGAP8XHS93H8dbFt")
#print(ans)
print(len(ans))
food_list = ans['foods']
print('food light len', len(food_list))
food_name_id_dict = {}
food_nutrient_dict = {}
nutrientName_list = []
for f in food_list:
    cols_interest = ['fdcId', 'description', 'dataType', 'brandOwner', 'brandName', 'foodCategory', 'servingSizeUnit', 'servingSize']
    for c in cols_interest:
        food_name_id_dict[c] = f.get(c, None)
    #print(food_name_id_dict, '\n')
    food_nutrients_list = f.get('foodNutrients', None)
    #print('number of nutrient for', f['fdcId'], len(food_nutrients_list))
    for nutrient in food_nutrients_list:
        nutrient_cols_interest = ['nutrientId', 'nutrientName', 'nutrientNumber', 'unitName', 'derivationCode', 'derivationDescription',
                                 'derivationId', 'value', 'foodNutrientSourceId', 'foodNutrientSourceCode', 'FoodNutrientSourceDescription',
                                 'rank']
        '''
        if nutrient.get('nutrientName') == 'Carbohydrate, by difference':
            for n in nutrient_cols_interest:
                food_nutrient_dict['fdcId'] = f.get('fdcId', None)
                food_nutrient_dict[n] = nutrient.get(n, None)
                food_name_id_dict[n] = nutrient.get(n, None)
        '''        
        nutrientName_list.append(nutrient.get('nutrientName'))
        food_nutrient_dict[nutrient['nutrientNumber']] = nutrient.get('nutrientName')
    #break
pd.DataFrame(food_name_id_dict, index=[0])

200
7
food light len 50


,fdcId,description,dataType,brandOwner,brandName,foodCategory,servingSizeUnit,servingSize
0,1999631,"Almond milk, unsweetened, plain, shelf stable",Foundation,None,None,Beverages,None,None


In [114]:
pd.DataFrame(food_nutrient_dict, index=[0])

,328,401,404,405,418,269,406,435,204,205,...,632,633,634,681,682,635,830,676.2,683,832
0,Vitamin D (D2 + D3),"Vitamin C, total ascorbic acid",Thiamin,Riboflavin,Vitamin B-12,"Sugars, total including NLEA",Niacin,"Folate, DFE",Total lipid (fat),"Carbohydrate, by difference",...,SFA 5:0,SFA 7:0,SFA 9:0,SFA 21:0,SFA 23:0,MUFA 12:1,TFA 20:1 t,MUFA 22:1 n-11,PUFA 22:3,TFA 18:2 t


In [117]:
print(len(set(nutrientName_list)))
nutrient_list_df = pd.DataFrame(food_nutrient_dict.items(), columns=['nutrientId', 'nutrientName']).sort_values(['nutrientName'])
nutrient_list_df.to_csv('./helpers/nutrient_list.csv', index=False)

190
